# Bounding Box Extraction
In this notebook we extract the bounding boxes (BBs) from the [WILDTRACK](https://openaccess.thecvf.com/content_cvpr_2018/papers/Chavdarova_WILDTRACK_A_Multi-Camera_CVPR_2018_paper.pdf) images using the coordinates provided in the dataset.

In [1]:
import numpy as np
import pandas as pd
import os
import cv2

from tqdm import tqdm
import time

#### 1. Creating Folders for Cropped BBs
In each camera directory create a new folder `croppedBB` in which we are going to store the cropped BBs. Execute the following code snippet only once.

In [2]:
cameras = ['C1','C2','C3','C4','C5','C6','C7']

parent_dir = '/home/ge93qew/WILDTRACK/Image_subsets/'
child_dir = 'croppedBB/'

for cam in cameras:
    path = os.path.join(parent_dir,cam,child_dir)
    isExist = os.path.exists(path)

    if not isExist:    
        os.makedirs(path)
        print(f"The new directory {path} is created!")

#### 2. Loading and Cropping the Images
Once the target folder has been created, we can crop the BBs. A cropped box represents one person and is associated with a `person_id`, `frame_id` and the corresponding box coordinates. We have 7 cameras with overlapping field of view each providing 400 frames. All the information and data are given in the WILDTRACK dataset. 

I made the following observations and adapted my code accordingly:
- There are cases where the ground truth BB of a person exceeds the image boundary, i.e. the BB cannot capture the person on the specific image. This is most probably because the BBs have been determined by projection using all 7 camera perspectives. My remedy was to just omit these samples and to log them in `broken_samples_log`.
- Surprisingly, there are images that obviously show more than 10 pedestrians that haven't been annotated, i.e. there are persons in some images for whom there are no BBs.
- Keep in mind that one cropped box may show more than 1 person.

In [7]:
cameras = ['C1','C2','C3','C4','C5','C6','C7']

parent_dir = '/home/ge93qew/WILDTRACK/Image_subsets/'
img_child_dir = 'img1/'
gt_child_dir = 'gt/gt.txt'
store_dir = 'croppedBB/'
broken_samples_log = ['img_dir, frame_id, cam_id, person_id, xmin, ymin, xmax, ymax']

for cam in cameras:
    gt_path = os.path.join(parent_dir,cam,gt_child_dir)
    img_path = os.path.join(parent_dir,cam,img_child_dir)
    store_path = os.path.join(parent_dir,cam,store_dir)
    print(f'gt_path: {gt_path},\nim_path: {img_path},\nst_path: {store_path}')
    data = pd.read_csv(gt_path, sep=",", header=None)
    data.columns = ["frame_id", "person_id", "xmin", "ymin","dx","dy","1.0","none","none","none"]   
    
    for i in tqdm(range(1,401)):
        bounding_boxes = data.loc[data['frame_id'] == i]
        
        person_ids = bounding_boxes.person_id
        frame_ids = bounding_boxes.frame_id
        xmins = bounding_boxes.xmin
        ymins = bounding_boxes.ymin
        dxs = bounding_boxes.dx
        dys = bounding_boxes.dy
        
        im_labels = sorted(os.listdir(img_path))
        img_dir = os.path.join(img_path,im_labels[i-1])
        img = cv2.imread(img_dir) # load colored images
        #print(img_dir)
        for bb in range(len(bounding_boxes)):
            xmin = xmins.iloc[bb]
            ymin = ymins.iloc[bb]
            dx = dxs.iloc[bb]
            dy = dys.iloc[bb]
            
            ## Labels
            gt_person_id = '{:04}'.format(person_ids.iloc[bb])
            gt_frame_id = '{:04}'.format(frame_ids.iloc[bb])
            gt_xmin = '{:04}'.format(xmin)
            gt_ymin = '{:04}'.format(ymin)
            gt_xmax = '{:04}'.format(xmin+dx)
            gt_ymax = '{:04}'.format(ymin+dy)
            
            if xmin in range(1920) and ymin in range(1080) and xmin+dx in range(1920) and ymin+dy in range(1080):
                cropped_image = img[ymin:ymin+dy, xmin:xmin+dx]
                #cv2.imwrite(f'{store_path}{gt_frame_id}-{cam}-{gt_person_id}-{im_labels[i-1]}', cropped_image)
            else:                              
                tmp_path = f'{img_dir},{gt_frame_id}-{cam}-{gt_person_id}-{gt_xmin}-{gt_ymin}-{gt_xmax}-{gt_ymax}'
                broken_samples_log.append(tmp_path)

gt_path: /home/ge93qew/WILDTRACK/Image_subsets/C1/gt/gt.txt,
im_path: /home/ge93qew/WILDTRACK/Image_subsets/C1/img1/,
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C1/croppedBB/


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:17<00:00, 22.69it/s]


gt_path: /home/ge93qew/WILDTRACK/Image_subsets/C2/gt/gt.txt,
im_path: /home/ge93qew/WILDTRACK/Image_subsets/C2/img1/,
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C2/croppedBB/


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:16<00:00, 24.58it/s]


gt_path: /home/ge93qew/WILDTRACK/Image_subsets/C3/gt/gt.txt,
im_path: /home/ge93qew/WILDTRACK/Image_subsets/C3/img1/,
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C3/croppedBB/


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:15<00:00, 26.09it/s]


gt_path: /home/ge93qew/WILDTRACK/Image_subsets/C6/gt/gt.txt,
im_path: /home/ge93qew/WILDTRACK/Image_subsets/C6/img1/,
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C6/croppedBB/


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:16<00:00, 24.56it/s]


gt_path: /home/ge93qew/WILDTRACK/Image_subsets/C7/gt/gt.txt,
im_path: /home/ge93qew/WILDTRACK/Image_subsets/C7/img1/,
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C7/croppedBB/


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:15<00:00, 26.18it/s]


Save the log file for the broken samples.

In [6]:
with open('/home/ge93qew/WILDTRACK/Image_subsets/broken-samples-log.txt', 'w') as f:
    for item in broken_samples_log:
        f.write("%s\n" % item)